## Bronze Layer: Ingesting Data to Databricks

In [0]:
import requests,json
from pyspark.sql import SparkSession

#defining the spark session
spark = SparkSession.builder.appName("JobETL_Bronze").getOrCreate()

# defining the API key
API_KEY = "278025380e0c67b179f77be267daf50fa06806f4911fc87dcfc3af7aa0dc79a3"

roles = ["Data Engineer", "Python Developer", "ETL Developer", "Spark Engineer", "Data Analyst"]
location = "India"
all_jobs = []

#Running a Loop over all the Job Roles
for role in roles:
    params={
        "engine":"google_jobs",
        "q":role, #"q": role → "q" yaha par role ka naam jayega (e.g., "Data Engineer")
        "location":location,
        "api_key":API_KEY
    }
    res = requests.get("https://serpapi.com/search.json", params=params)
    jobs = res.json().get("jobs_results", [])

    # Print the whole response to see what's inside
    # print(json.dumps(res.json(), indent=2))   
     
    # print(jobs)   getting API response

    for job in jobs:
        job["search_role"] = role
        all_jobs.extend(jobs)

# bronze_df = spark.read.json(spark.sparkContext.parallelize([json.dumps(job) for job in all_jobs]))
# bronze_df.write.format("delta").option("mergeSchema", "true").mode("append").save("/mnt/lakehouse/bronze/jobs_raw")

# ✅ Convert Python list of dicts directly into Spark DF
bronze_df = spark.createDataFrame(all_jobs)

# ✅ Save to Delta (use managed table instead of DBFS path in free version)
bronze_df.write.format("delta").mode("overwrite").saveAsTable("bronze_jobs_raw")

In [0]:
display(bronze_df)

## Silver Layer: Cleaning and Structuring the Data

In [0]:
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("JobETL_Silver").getOrCreate()

#reading bronze layer
df = spark.read.table("bronze_jobs_raw")

#cleaning and structuring the data
silver_df =  df.selectExpr(
    "title",
    "company_name",
    "location",
    "description",
    "job_id",
    "detected_extensions.posted_at as posted_at",
    "search_role"
).dropna(subset=["title", "company_name", "location"])

In [0]:
display(silver_df)

In [0]:
silver_df = silver_df.withColumn("company_name", trim(upper(col("company_name"))))
silver_df = silver_df.dropna(subset=["posted_at"])
silver_df.write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable("silver_jobs")
display(silver_df)

## Golend Layer: Generate KPI Tables

In [0]:
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("JobETL_Gold").getOrCreate()

#reading silver layer
df = spark.read.table("silver_jobs")